In [12]:
pip install tensorflow python-dotenv pandas numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


## Import Necessary Libraries

In [13]:
import os
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv

## Load Data

In [15]:
load_dotenv()  # Load environment variables from .env file
jwt_token = os.getenv('JWT_TOKEN')

headers = {'Authorization': f'Bearer {jwt_token}'}

api_urls = {
    'interactions': 'http://161.97.109.65:3000/api/interactions/download',
    'users': 'http://161.97.109.65:3000/api/users/download',
    'products': 'http://161.97.109.65:3000/api/products/download'
}

def fetch_data(url, headers):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raises an HTTPError for bad requests
        return pd.DataFrame(response.json())
    except requests.RequestException as e:
        print(f'Failed to fetch data from {url}: {str(e)}')
        return pd.DataFrame()

# Fetch data from APIs
users = fetch_data(api_urls['users'], headers)
products = fetch_data(api_urls['products'], headers)
interactions = fetch_data(api_urls['interactions'], headers)

if not users.empty and not products.empty and not interactions.empty:
    print("Data fetched successfully.")
else:
    print("Data fetching failed, check errors and retry.")
    # Optionally, add logic to halt further processing if data is crucial


Data fetched successfully.


## Data Preprocessing

In [16]:
if 'user_id' in interactions.columns and 'product_id' in interactions.columns:
    print("Data columns verified.")
else:
    raise ValueError("Required columns are missing from the interactions data.")

# Further data processing steps here

ValueError: Required columns are missing from the interactions data.

## Define Model

In [ ]:
# Define the model
user_input = Input(shape=(1,))
user_embedding = Embedding(len(users['user_id'].unique()), 50, input_length=1)(user_input)
user_vec = Flatten()(user_embedding)

product_input = Input(shape=(1,))
product_embedding = Embedding(len(products['product_id'].unique()), 50, input_length=1)(product_input)
product_vec = Flatten()(product_embedding)

dot_product = Dot(axes=1)([user_vec, product_vec])
model = Model([user_input, product_input], dot_product)
model.compile(optimizer='adam', loss='mean_squared_error')


## Train Model

In [ ]:
# Assuming 'train' and 'test' datasets are already split and preprocessed
x_train = [train['user'].values, train['product'].values]
y_train = train['interaction_value'].values

history = model.fit(x_train, y_train, epochs=20, verbose=1, validation_split=0.2)


## Try Model

In [ ]:
def recommend_products(user_id, model, interactions, user2user_encoded, product2product_encoded, productencoded2product, products, top_n=30):
    user_encoded = user2user_encoded[user_id]
    product_ids = list(product2product_encoded.values())
    user_product_array = np.hstack(([[user_encoded]] * len(product_ids), np.array(product_ids).reshape(-1, 1)))

    predictions = model.predict([user_product_array[:, 0], user_product_array[:, 1]])
    predictions = predictions.flatten()
    top_indices = predictions.argsort()[-top_n:][::-1]
    recommended_product_ids = [productencoded2product[x] for x in top_indices]
    recommended_products = products[products['product_id'].isin(recommended_product_ids)]
    return recommended_products

user_id = 23
recommended_products = recommend_products(user_id, model, interactions, user2user_encoded, product2product_encoded, productencoded2product, products)
print(f"Recommended products for user {user_id}:")
print(recommended_products)